In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
#!pip install tensorflow==2.0.0-alpha0 
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
tf.__version__

'2.0.0-dev20190602'

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data("mnist.npz")
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [5]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
    raw_perturb = tf.gradients(loss, images)[0]
    normalized_per=tf.nn.l2_normalize(raw_perturb, axis=[1, 2, 3])
    perturb =0.01 * tf.stop_gradient(normalized_per)
    perturb_image_inputs = images + perturb
    perturb_predictions = model(perturb_image_inputs)
    perturb_loss = loss_object(labels, perturb_predictions)
    all_loss = loss + perturb_loss
  gradients = tape.gradient(all_loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(all_loss)
  train_accuracy(labels, predictions)

In [9]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [10]:
EPOCHS = 20

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

Epoch 1, Loss: 0.280753493309021, Accuracy: 95.8183364868164, Test Loss: 0.05578523874282837, Test Accuracy: 98.22000122070312
Epoch 2, Loss: 0.18032361567020416, Accuracy: 97.32083129882812, Test Loss: 0.05583876371383667, Test Accuracy: 98.17500305175781
Epoch 3, Loss: 0.13333258032798767, Accuracy: 98.00999450683594, Test Loss: 0.05759581923484802, Test Accuracy: 98.1199951171875
Epoch 4, Loss: 0.10752709954977036, Accuracy: 98.38958740234375, Test Loss: 0.06005034223198891, Test Accuracy: 98.0625
Epoch 5, Loss: 0.089377760887146, Accuracy: 98.65766906738281, Test Loss: 0.06132921203970909, Test Accuracy: 98.13600158691406
Epoch 6, Loss: 0.07705852389335632, Accuracy: 98.84249877929688, Test Loss: 0.06034369021654129, Test Accuracy: 98.22999572753906
Epoch 7, Loss: 0.06748607009649277, Accuracy: 98.98428344726562, Test Loss: 0.06186548247933388, Test Accuracy: 98.27285766601562
Epoch 8, Loss: 0.060236431658267975, Accuracy: 99.09166717529297, Test Loss: 0.062457744032144547, Test Ac